In [ ]:
# Install required dependencies

!pip install crewai==0.28.8 crewai_tools==0.1.6

https://docs.crewai.com/introduction

In [ ]:
# waring control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import CrewAI components for building agents and tasks

from crewai import Agent, Task, Crew

In [ ]:
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id = "HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token = "Enter your Hugging face hun api key",
    task = "text-generation",
)

/usr/local/lib/python3.11/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [ ]:
# import os
# # from .utils import get_openai_api_key

# # openai_api_key = get_openai_api_key()
# os.environ["OPENAI_API_KEY"] = openai_api_key
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# not required to pass llm to the agent funtion if you are using OpenAI API

# Creating Agents

Agent is and autonomous unit that can <br>
* Perform specific tasks
* Make decisions based on its role and goal
* Use tools to accomplish objectives
* Communicate and collaborate with other agents
* Maintain memory of interactions
* Delegate tasks when allowed

First we creat Agents, where the task of the agents is.<br>
 1. to research about the given topic (Planner).
 2. to wrtie about the searched content in a blog (wrtie).
 3. to edit the writen content and make it more interesting for the user(editor).

## Agent: Planner

In [ ]:
Planner = Agent(
    role  = "Content Planner",
    goal = "Plan engaging and factually accurate content on {topic_given}",
    # role play telling the agent how it should thing
    backstory = "You're working on planning a blog article"
                "about the topic: {topic_given}."
                "You collect information that helps the"
                "audience learn something "
                "and mark informed decisions."
                "Your work is basis for"
                "the Content Writer to write an article on this topic.",
    allow_delegation = False,
    verbose = True,
    llm = llm,
)

## Agent: Writer

In [ ]:
Writer = Agent(
    role = "Content writer",
    goal = "Write insightful and factually accurate"
           "opinion piece about the topic: {topic_given}",
    backstory = "You're working on writing"
                "a new opinion piece about the topic: {topic_given}."
                "You base your writting on the work of"
                "the content Planner, who provides an outline"
                "and relevant context about the topic."
                "You follow the main objectives and "
                "direction of the outline,"
                "as provide by the Content Planer."
                "You also provide objective and impartial insights"
                "and back them up with inforamtion"
                "provide by the Content Planer.",
    allow_delegation = False,
    verbose = True,
    llm = llm,

)

## Agent: Editor

In [ ]:
Editor = Agent(
    role = "Editor",
    goal = "Edit a given blog post to align with"
           "the writing style of the organization.",
    backstory = "You are an editor who recevies a blog post"
                "from the Content Writer."
                "Your task is to review the blog post"
                "to ensure that it follows the best practices of a Journalist,"
                "Provides balanced viewpoints"
                "when providing opinions or assertions,"
                "and also avoids major controversial topics"
                "or opinions when possible.",
    allow_delegation = False,
    verbose = True,
    llm = llm,
)

# Creating Tasks

Second step is to creat the task, where the each task have a specifice assignment cmpleted by an Agent.<br>
Tasks provide all necessary details for execution, such as a description, the agent responsible, required tools.

## Task: Plan

In [ ]:
plan = Task(
    description = (
        "1. Prioritize the latest trends, key players, "
            "and noteworthy new on {topic_given}. \n"
        "2. Identify the target audience, considering"
            "their interests and pain points. \n"
        "3. Develop a detailed content outline inculding"
            "an introduction, key points, and a call to action."
        "4. Include SEO keywords and relevent data or sources."
    ),
    expected_output = "A Comprehensive content plan document "
                      "with an outline, audience analysis, "
                      "SEO keywords, and resources. ",
    agent = Planner,
)

## Task: write

In [ ]:
write = Task(
    description = (
        "1. Use the content plan to craft a compelling "
            "blog post on {topic_given}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		    "3. Sections are properly named in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output = "A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent = Writer,
)

## Task: Edit

In [ ]:
edit = Task(
    description=(
                "Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."
    ),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=Editor
)

# Creating Crew

A crew in crewAI represents a collaborative group of agents working together to achieve a set of tasks. Each crew defines the strategy for task execution, agent collaboration, and the overall workflow.

In [ ]:
crew = Crew(
    agents = [Planner, Writer, Editor],
    tasks = [plan, write, edit],
    verbose = 2
)

In [ ]:
# running the crew to get the output for the given topic
Topic = "LLM"
# Topic = "India"
result = crew.kickoff(inputs = {"topic_given": Topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy new on LLM. 
2. Identify the target audience, consideringtheir interests and pain points. 
3. Develop a detailed content outline inculdingan introduction, key points, and a call to action.4. Include SEO keywords and relevent data or sources.


> Entering new CrewAgentExecutor chain...
You are Content Planner. You're working on planning a blog articleabout the topic: LLM.You collect information that helps theaudience learn something and mark informed decisions.Your work is basis forthe Content Writer to write an article on this topic.
Your personal goal is: Plan engaging and factually accurate content on LLMTo give my best complete final answer to the task use the exact following format:

Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.
Your final answer must be the great and the most complete as possible, it m

In [ ]:
# Diaplying the result in an markdown in the notebook
from IPython.display import Markdown
Markdown(result)

my best complete final answer to the task.
Your final answer must be the great and the most complete as possible, it must be outcome described.

I MUST use these formats, my job depends on it!
Current Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.

This is the expect criteria for your final answer: A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs. 
 you MUST return the actual complete content as the final answer, not a summary.

This is the context you're working with:
# **Unlocking the Power of Large Language Models (LLMs)**

## **Introduction**

In the rapidly evolving landscape of artificial intelligence, one technology has emerged as a game-changer: Large Language Models (LLMs). These models, powered by deep learning and natural language processing (NLP), are transforming industries and reshaping our understanding of human language. But what exactly are LLMs, and why are they causing such a stir? Let's dive in.

LLMs are neural networks trained on vast amounts of text data, enabling them to understand, generate, and interact with human language in ways that were once unimaginable. They form the backbone of many AI applications we use daily, from voice assistants like Siri and Alexa to language translation tools and text prediction features on our smartphones.

The significance of LLMs lies in their ability to adapt to various tasks and domains. Unlike traditional NLP models that require task-specific training, LLMs can be fine-tuned or instructed to perform a wide range of tasks, from sentiment analysis and machine translation to question answering and text summarization. This versatility has sparked a wave of innovation, with tech giants and startups alike racing to develop the next big LLM.

## **Understanding LLMs: Architecture, Training, and Applications**

At their core, LLMs are based on transformer architecture, introduced in the groundbreaking paper "Attention Is All You Need" by Vaswani et al. (2017). Transformers use self-attention mechanisms to weigh the importance of input words, enabling them to capture complex dependencies in text data. This architecture has proven remarkably effective for various NLP tasks.

Training LLMs involves feeding them massive amounts of text data, allowing them to learn patterns, grammar, and semantics. The more data they're trained on, the better they perform. For instance, BERT (Bidirectional Encoder Representations from Transformers), developed by Google, was trained on the entire Wikipedia and BookCorpus, totaling 3.3 billion words.

The applications of LLMs are vast and varied. They power chatbots and virtual assistants, generate human-like text for creative writing and content creation, and even help in scientific research, as demonstrated by Facebook AI's Galactica model. However, their true potential lies in their ability to adapt and evolve, as we'll explore next.

## **Latest Trends in LLMs: Foundation Models, Instruction Tuning, and Multimodal Learning**

One of the most exciting trends in LLMs is the development of foundation models. These models, like BERT, RoBERTa, and T5, are trained on diverse, unlabeled data and can be adapted for various downstream tasks with minimal task-specific training. This approach has led to significant improvements in performance and efficiency.

Another trend is instruction tuning, where LLMs are fine-tuned to follow instructions and perform tasks better. For example, Google's Coté model is trained to explain its reasoning, making it more transparent and interpretable. This trend aligns with the growing demand for AI systems that are not only powerful but also explainable and fair.

Multimodal learning is another burgeoning field, integrating LLMs with other modalities like vision and audio. Models like DALL-E 2 can generate images from textual descriptions, blurring the lines between different types of AI. This trend opens up new possibilities for creative applications, such as generating music or designing products based on textual inputs.

## **Key Players in LLM Development**

The race to develop the most advanced LLMs is being led by a mix of research institutions, tech companies, and startups. Stanford University, MIT, and UC Berkeley are at the forefront of LLM research, publishing groundbreaking papers and nurturing talent in the field.

Tech giants like Google, Facebook, and Microsoft are also heavily invested in LLM development. Google's BERT and T5 models have set the bar for state-of-the-art performance, while Facebook's RoBERTa has pushed the boundaries of instruction tuning. Microsoft's Turing NLG and IBM's Project Debater are other notable examples of industry-led LLM development.

Startups are also making significant strides in the LLM space. Hugging Face, Cohere, and AI21 Labs are developing innovative LLMs and tools that democratize access to these powerful models. Their work is crucial for fostering a vibrant ecosystem of LLM applications and use cases.

## **Noteworthy News and Innovations**

The past year has seen several remarkable innovations in LLMs. DALL-E 2, developed by AI lab, can generate intricate images from textual descriptions, demonstrating the power of multimodal learning. Galactica, from Facebook AI, is a LLM designed specifically for scientific research and writing, highlighting the potential of LLMs in specialized domains.

Coté, developed by Google DeepMind, is a LLM that can explain its reasoning, marking a significant step towards interpretable AI. This innovation aligns with the growing demand for AI systems that are not only powerful but also transparent and accountable.

## **Conclusion: The Future of LLMs**

The rapid advancements in LLMs are reshaping our world, from how we interact with technology to how we create and consume content. As LLMs continue to evolve, so too will their applications and impact. But with great power comes great responsibility. It's crucial that we develop LLMs ethically, ensuring they are fair, unbiased, and respect user privacy.

As we stand on the cusp of a new era of AI, we invite you to explore LLMs further. Whether you're a tech enthusiast, a data scientist, or a business professional, there's much to learn and gain from these powerful models. Stay informed, stay curious, and be part of shaping the future of LLMs.

**Recommended Resources:**

- "The State of AI Report 2022" by Stanford University.
- "AI Index Report 2022" by the Stanford Institute for Human-Centered AI.
- Towards Data Science (towardsdatascience.com)
- KDnuggets (kdnuggets.com)
- AI Trends (aitrends.com)

**What do you think about the future of LLMs? Share your thoughts in the comments below!**

Begin! This is VERY important to you, use the tools available and give your best Final Answer, your job depends on it!

Thought: 
- AI Trends (aitrends the future of LLMs, and the c